## Test Example

The file `test_one.py` uses the `sqlite3` module to create the database depicted in the figure below (repeated throughout for convenience).

![Entity-relationship diagram](../erd.png)

In [1]:
# run the script and import vars
from test_one import *

The cursor is called `cur` and the database connection is called `con`. Both are active in the notebook's kernel now.

In addition to the tables and relations shown, ten patients with various conditions have been entered in the database. 

The `sqlite3` module allows definition of a query as a Python string, which can be executed with the cursor as follows:

In [2]:
query = """SELECT * FROM patient"""
res = cur.execute(query)
res.fetchall()

[(1, 'John', '1982 Well St'),
 (2, 'Jill', '1983 Well St'),
 (3, 'Jasper', '1984 Well St'),
 (4, 'Janine', '1985 Well St'),
 (5, 'Kato', '10 Willow St'),
 (6, 'Leonard', '19 Marcus St'),
 (7, 'Lillian', '233 First St'),
 (8, 'Mathilda', '1985 Well St'),
 (9, 'Nancy', '1982 Well St'),
 (10, 'Ophelia', '1982 Well St')]

1. Write a Python function to take a SQL statement as string and return all of the results.

In [3]:
def fetch_all(q):
    """
    Fetch all results from database for query `q`
    """
    res = cur.execute(q)
    return res.fetchall()

Referencing the figure, write SQL statements to achieve the following goals (note, the table names and attributes in the figure match those in the database exactly). Execute the statements using your Python function from above.

![Entity-relationship diagram](../erd.png){width=75%}

1. How many different conditions are there, and what are they?

In [4]:
query = """SELECT * FROM condition"""
fetch_all(query)

[(1, 'Fever'),
 (2, 'Cold'),
 (3, 'Plague'),
 (4, 'Tachycardia'),
 (5, 'Fatigue'),
 (6, "Munchausen's")]

2. How many patients suffer from 'Plague'?

In [5]:
query = """SELECT COUNT(*) FROM diagnosis WHERE condition_id = 3"""
fetch_all(query)

[(4,)]

3. Write and execute a Python function that uses your first function to compute the difference between patients with Plague and those with either Fever or Cold. Return the result as a statment like "The difference is ..."

In [11]:
def get_difference():
    """
    Compute difference between patients with Plague and those with Fever or Cold.
    """
    n_plague = fetch_all("""SELECT COUNT(*) FROM diagnosis WHERE condition_id = 3""")[0][0]
    n_fever_cold = fetch_all("""SELECT COUNT(*) FROM diagnosis WHERE condition_id IN (1,2)""")[0][0]
    return "The difference is {}".format(n_plague-n_fever_cold)

In [12]:
get_difference()

'The difference is 2'

4. Develop a SQL statement to return the names and addresses of patients with Plague.

In [6]:
query = """SELECT patient_name, patient_address
FROM (patient INNER JOIN diagnosis on diagnosis.patient_id = patient.patient_id)
WHERE diagnosis.condition_id=3"""
fetch_all(query)

[('Ophelia', '1982 Well St'),
 ('Nancy', '1982 Well St'),
 ('John', '1982 Well St'),
 ('Jill', '1983 Well St')]

5. Add a column to the last result to show how they are being medicated.

In [13]:
query = """
SELECT patient_name, patient_address, med_name FROM
    (SELECT patient_name, patient_address, med_id
    FROM (patient INNER JOIN diagnosis on diagnosis.patient_id = patient.patient_id)
    WHERE diagnosis.condition_id=3) 
    AS q1
INNER JOIN medication on medication.med_id = q1.med_id
"""
fetch_all(query)

[('Ophelia', '1982 Well St', 'Doxycycline'),
 ('Nancy', '1982 Well St', 'Doxycycline'),
 ('John', '1982 Well St', 'Doxycycline'),
 ('Jill', '1983 Well St', 'Doxycycline')]

Remember to close the database connection:

In [7]:
con.close()

Data model discussion questions: 
-  Is a single patient able to have more than one "condition"?
-  Is a patient able to have a "condition" without a "diagnosis"?
-  Is the doctor-patient relationship well designed in the model provided? If not what are some of the limitations? How would you recommend correcting this?
-  Other recommendations to improve the model?

![Entity-relationship diagram](../erd.png){width=90%}

Acceptable Answers: 

-  Yes
-  No  
-  No, it's not, because a doctor is directly associated with a patient. A potential solution would be to include a separate doctor table and a bridge entity to enable a more realistic situation in which each patient has many doctors and each doctor has many patients. 
-  Open ended, but for example, the diagnosis table could be improved by adding the doctor_id to track which doctor made each diagnosis.